## Gold: Exportação Vinhos

In [0]:
%run "../1 - Setup/Ingestao_bibliotecas_padrao"


In [0]:
from pyspark.sql.functions import col

In [0]:
tabela_silver = spark.table("vitivinicultura.silver_exportacao.exportacao_vinhos")

In [0]:
colunas_anos = [str(ano) for ano in range(1970, 2023)]

# 🧩 2️⃣ Cria expressão para "unpivotar" litros e usd separadamente
expr_litros = "stack({n}, {pairs}) as (ano, litros)".format(
    n=len(colunas_anos),
    pairs=", ".join([f"'{ano}', `{ano}_litros`" for ano in colunas_anos])
)

expr_usd = "stack({n}, {pairs}) as (ano, usd)".format(
    n=len(colunas_anos),
    pairs=", ".join([f"'{ano}', `{ano}_usd`" for ano in colunas_anos])
)

# 🧩 3️⃣ Gera dois DataFrames unpivotados
df_litros = tabela_silver.selectExpr("`País`", expr_litros)
df_usd = tabela_silver.selectExpr("`País`", expr_usd)


In [0]:
df_litros.createOrReplaceTempView("exportacao_vinhos_litros")
df = spark.sql("""
    SELECT 
        `País`                                                       AS PAIS, 
        ANO, 
        TO_DATE(CONCAT('01-01-', CAST(ano AS STRING)), 'dd-MM-yyyy') AS DATA,
        litros                                                       AS LITROS
    FROM exportacao_vinhos_litros
""")
df.createOrReplaceTempView("exportacao_vinhos_litros")

In [0]:
df_usd.createOrReplaceTempView("exportacao_vinhos_usd")
df = spark.sql("""
    SELECT 
        `País`                                                       AS PAIS, 
        ANO, 
        TO_DATE(CONCAT('01-01-', CAST(ano AS STRING)), 'dd-MM-yyyy') AS DATA,
        usd                                                          AS VALOR_USD
    FROM exportacao_vinhos_usd
""")
df.createOrReplaceTempView("exportacao_vinhos_usd")

In [0]:
df_litros.createOrReplaceTempView("paises")
df = spark.sql("""
    SELECT DISTINCT `País` as PAIS
    FROM paises
    WHERE 'País' IS NOT NULL
""")
df.createOrReplaceTempView("paises_distinct")

In [0]:
df = spark.sql("""
    SELECT ROW_NUMBER() OVER (ORDER BY PAIS ASC) AS ID_PAIS,
    PAIS
    FROM paises_distinct
    WHERE PAIS IS NOT NULL
""")
df.createOrReplaceTempView("paises")

In [0]:
df = spark.sql("""
    SELECT 
        E.PAIS,
        E.ANO,
        E.DATA,
        CASE WHEN E.LITROS = 'None' THEN 0    ELSE E.LITROS END AS LITROS,
        CASE WHEN P.VALOR_USD = 'None' THEN 0 ELSE P.VALOR_USD END AS VALOR_USD
    FROM exportacao_vinhos_litros AS E
    INNER JOIN exportacao_vinhos_usd AS P ON E.PAIS = P.PAIS AND E.ANO = P.ANO
""")
df.createOrReplaceTempView("exportacao_vinhos")

In [0]:
df = spark.sql("""
    SELECT 
        ROW_NUMBER() OVER (ORDER BY E.PAIS, E.ANO ASC) AS ID_EXPORTACAO_VINHO,
        P.ID_PAIS,
        E.PAIS,
        E.ANO,
        E.DATA,
        E.LITROS,
        E.VALOR_USD
    FROM exportacao_vinhos AS E
    INNER JOIN paises AS P ON E.PAIS = P.PAIS
""")
df = add_metadata_cols(df)
df.createOrReplaceTempView("vinhos_exportados_final")

In [0]:
 #Cria tabela gerenciada no Unity Catalog
spark.sql("""
CREATE TABLE IF NOT EXISTS vitivinicultura.gold_exportacao.exportacao_vinhos
""")
# Escrita no formato Delta Lake
df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("vitivinicultura.gold_exportacao.exportacao_vinhos")

